In [1]:
#imports
import copy
import random
import numpy as np
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from sklearn.metrics import accuracy_score

In [ ]:
#globals
def fix_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

seed = 0
fix_seed(seed)

num_clients = 10
num_rounds = 5
batch_size = 64
lr = 0.01
criterion = F.nll_loss

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [3]:
#utils

#dataloader utils
class NumpyDataset(Dataset):
    """This class allows you to convert numpy.array to torch.Dataset"""

    def __init__(self, x, y=None, transform=None, return_idx=False):
        self.x = x
        self.y = y
        self.transform = transform
        self.return_idx = return_idx

    def __getitem__(self, index):
        x = self.x[index]
        if self.y is not None:
            y = self.y[index]

        if self.transform is not None:
            x = self.transform(x)

        if not self.return_idx:
            if self.y is not None:
                return x, y
            else:
                return x
        else:
            if self.y is not None:
                return index, x, y
            else:
                return index, x

    def __len__(self):
        """get the number of rows of self.x"""
        return len(self.x)

#client utils
def default_local_train_for_client(self, local_epoch, criterion, trainloader, optimizer):
    running_loss = 0.0
    for _ in range(local_epoch):
        for data in trainloader:
            _, x, y = data
            x = x.to(self.device)
            y = y.to(self.device).to(torch.int64)

            optimizer.zero_grad()
            loss = criterion(self(x), y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

    return running_loss

class RoundDecimal(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input, n_digits):
        ctx.save_for_backward(input)
        ctx.n_digits = n_digits
        return torch.round(input * 10**n_digits) / (10**n_digits)

    @staticmethod
    def backward(ctx, grad_output):
        grad_input = grad_output.clone()
        return torch.round(grad_input * 10**ctx.n_digits) / (10**ctx.n_digits), None

torch_round_x_decimal = RoundDecimal.apply

def initialize_global_logit(len_public_dataloader, output_dim, device):
    return torch.ones((len_public_dataloader, output_dim)).to(device) * float("inf")

#FedMD utils
def accuracy_torch_dataloader(model, dataloader, device="cpu", xpos=1, ypos=2):
    """Calculates the accuracy of the model on the given dataloader
    Args:
        model (torch.nn.Module): model to be evaluated
        dataloader (torch.DataLoader): dataloader to be evaluated
        device (str, optional): device type. Defaults to "cpu".
        xpos (int, optional): the positional index of the input in data. Defaults to 1.
        ypos (int, optional): the positional index of the label in data. Defaults to 2.
    Returns:
        float: accuracy
    """
    in_preds = []
    in_label = []
    with torch.no_grad():
        for data in dataloader:
            inputs = data[xpos]
            labels = data[ypos]
            inputs = inputs.to(device)
            labels = labels.to(device).to(torch.int64)
            outputs = model(inputs)
            in_preds.append(outputs)
            in_label.append(labels)
        in_preds = torch.cat(in_preds)
        in_label = torch.cat(in_label)

    return accuracy_score(
        np.array(torch.argmax(in_preds, axis=1).cpu()), np.array(in_label.cpu())
    )

In [ ]:
#CIFAR10 dataset
def prepare_dataloader(num_clients, myid, train=True, path="."):
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.4914, 0.4821, 0.4465], std=[0.2470, 0.2435, 0.2616])])
    
    if train:
        dataset = datasets.CIFAR10(path, train=True, download=True, transform=transform)
        idxs = list(range(len(dataset.data)))
        random.shuffle(idxs)
        idx = np.array_split(idxs, num_clients, 0)[myid - 1]
        dataset.data = dataset.data[idx]
        dataset.targets = np.array(dataset.targets)
        dataset.targets = dataset.targets[idx]
        train_loader = torch.utils.data.DataLoader(
            NumpyDataset(
                x=dataset.data,
                y=dataset.targets,
                transform=transform,
                return_idx=True,
            ),
            batch_size=batch_size,
        )
        return train_loader
    else:
        dataset = datasets.CIFAR10(path, train=False, download=True, transform=transform)
        test_loader = torch.utils.data.DataLoader(
            NumpyDataset(
                x=dataset.data,
                y=dataset.targets,
                transform=transform,
                return_idx=True,
            ),
            batch_size=batch_size,
        )
        return test_loader

dataloaders = [prepare_dataloader(num_clients + 1, c) for c in range(num_clients + 1)]
public_dataloader = dataloaders[0]
private_dataloaders = dataloaders[1:]
test_dataloader = prepare_dataloader(num_clients, -1, train=False)

In [5]:
#client
class FedMDClient(torch.nn.Module):
    def __init__(
        self,
        model,
        public_dataloader,
        user_id=0,
        output_dim=1,
        batch_size=8,
        base_loss_func=nn.CrossEntropyLoss(),
        consensus_loss_func=nn.L1Loss(),
        round_decimal=None,
        device="cpu",
    ):
        super(FedMDClient, self).__init__()
        self.user_id = user_id
        self.model = model
        self.public_dataloader = public_dataloader
        self.batch_size = batch_size
        self.base_loss_func = base_loss_func
        self.consensus_loss_func = consensus_loss_func
        self.round_decimal = round_decimal
        self.device = device

        self.predicted_values_of_server = None

        len_public_dataloader = len(self.public_dataloader.dataset)
        self.logit2server = initialize_global_logit(
            len_public_dataloader, output_dim, self.device
        )

    def forward(self, x):
        return self.model(x)

    def train(self):
        self.model.train()

    def eval(self):
        self.model.eval()

    def backward(self, loss):
        loss.backward()

    def upload(self):
        for data in self.public_dataloader:
            idx = data[0]
            x = data[1]
            x = x.to(self.device)
            self.logit2server[idx, :] = self(x).detach()

        if self.round_decimal is None:
            return self.logit2server
        else:
            return torch_round_x_decimal(self.logit2server, self.round_decimal)

    def download(self, predicted_values_of_server):
        self.predicted_values_of_server = predicted_values_of_server

    def local_train(self, local_epoch, criterion, trainloader, optimizer):
        return default_local_train_for_client(
            self, local_epoch, criterion, trainloader, optimizer
        )

    def approach_consensus(self, consensus_optimizer):
        running_loss = 0

        for data in self.public_dataloader:
            idx = data[0]
            x = data[1].to(self.device)
            y_consensus = self.predicted_values_of_server[idx, :].to(self.device)
            consensus_optimizer.zero_grad()
            y_pred = self(x)
            loss = self.consensus_loss_func(y_pred, y_consensus)
            loss.backward()
            consensus_optimizer.step()
            running_loss += loss.item()

        return running_loss

In [6]:
#sever
class FedMDServer(torch.nn.Module):
    def __init__( self, clients, server_model=None, server_id=0, device="cpu" ):
        super(FedMDServer, self).__init__()
        self.clients = clients
        self.server_id = server_id
        self.server_model = server_model
        self.num_clients = len(clients)
        self.device = device
        self.uploaded_logits = []

    def forward(self, x):
        if self.server_model is not None:
            return self.server_model(x)
        else:
            return None

    def train(self):
        self.server_model.train()

    def eval(self):
        self.server_model.eval()
    
    def action(self):
        self.receive()
        self.update()
        self.distribute()

    def receive(self):
        self.uploaded_logits = [client.upload() for client in self.clients]

    def update(self):
        len_clients = len(self.clients)
        self.consensus = self.uploaded_logits[0] / len_clients
        for logit in self.uploaded_logits[1:]:
            self.consensus += logit / len_clients

    def distribute(self):
        """Distribute the logits of public dataset to each client."""
        for client in self.clients:
            client.download(self.consensus)


In [22]:
#FedMD
class FedMD(torch.nn.Module):
    def __init__(
        self,
        server,
        clients,
        public_dataloader,
        local_dataloaders,
        criterion,
        client_optimizers,
        test_dataloader=None,
        server_optimizer=None,
        num_communication=1,
        device="cpu",
        consensus_epoch=1,
        revisit_epoch=1,
        transfer_epoch_public=1,
        transfer_epoch_private=1,
        server_training_epoch=1,
        custom_action=lambda x: x,
    ):
        super(FedMD, self).__init__()
        self.server = server
        self.clients = clients
        self.public_dataloader = public_dataloader
        self.local_dataloaders = local_dataloaders
        self.test_dataloader = test_dataloader
        self.criterion = criterion
        self.num_communication = num_communication
        self.device = device
        self.client_optimizers = client_optimizers
        self.server_optimizer = server_optimizer
        self.consensus_epoch = consensus_epoch
        self.revisit_epoch = revisit_epoch
        self.transfer_epoch_public = transfer_epoch_public
        self.transfer_epoch_private = transfer_epoch_private
        self.server_training_epoch = server_training_epoch
        self.custom_action = custom_action
        self.epoch = 0
        self.client_num = len(clients)


    def train_client(self, epoch=1, public=True):
        """
        Train local models with the local datasets or the public dataset.
        input: public (bool, optional): Train with the public dataset or the local datasets.
        output: List[float]: a list of average loss of each clients.
        """
        loss_on_local_dataset = []
        for client_idx in range(self.client_num):
            if public:
                trainloader = self.public_dataloader
            else:
                trainloader = self.local_dataloaders[client_idx]

            running_loss = self.clients[client_idx].local_train(
                epoch, self.criterion, trainloader, self.client_optimizers[client_idx]
            )

            loss_on_local_dataset.append(copy.deepcopy(running_loss / len(trainloader)))

        return loss_on_local_dataset


    def train_server(self):
        if self.server_optimizer is None:
            return 0.0

        running_loss = 0.0
        for data in self.public_dataloader:
            _, x, y = data
            x = x.to(self.device)
            y = y.to(self.device).to(torch.int64)

            self.server_optimizer.zero_grad()
            loss = self.criterion(self.server(x), y)
            loss.backward()
            self.server_optimizer.step()

            running_loss += loss.item()

        running_loss /= len(self.public_dataloader)

        return running_loss


    def score(self, dataloader, only_local=False):
        """
        Returns the performance on the given dataset.
        input: dataloader (torch.utils.data.DataLoader): a dataloader
               only_local (bool): show only the local results
        output: Dict[str, int]: performance of global model and local models
        """

        clients_score = [
            accuracy_torch_dataloader(client, dataloader, device=self.device)
            for client in self.clients
        ]

        if only_local:
            return {"clients_score": clients_score}
        else:
            server_score = accuracy_torch_dataloader(
                self.server, dataloader, device=self.device
            )
            return {"server_score": server_score, "clients_score": clients_score}


    def local_score(self):
        """
        Returns the local performance of each clients.
        output: Dict[str, int]: performance of global model and local models
        """
        local_score_list = []
        for client, local_dataloader in zip(self.clients, self.local_dataloaders):
            temp_score = accuracy_torch_dataloader(
                client, local_dataloader, device=self.device
            )
            local_score_list.append(temp_score)

        return {"clients_score": local_score_list}


    def transfer_phase(self, logging):
        # Transfer
        for i in range(1, self.transfer_epoch_public + 1):
            loss_public = self.train_client(public=True)
            #print(f"epoch {i} (public - pretrain): {loss_public}")
            logging["loss_client_public_dataset_transfer"].append(loss_public)
        
        if self.public_dataloader is not None:
            acc_pub = self.score(
                self.public_dataloader, self.server_optimizer is None
            )
            #print("acc on public dataset: ", acc_pub)
            logging["acc_pub"].append(copy.deepcopy(acc_pub))

        for i in range(1, self.transfer_epoch_private + 1):
            loss_local = self.train_client(public=False)
            #print(f"epoch {i} (local - pretrain): {loss_local}")
            logging["loss_client_local_dataset_transfer"].append(loss_local)
        
        if self.public_dataloader is not None:
            acc_pub = self.score(
                self.public_dataloader, self.server_optimizer is None
            )
            #print("acc on public dataset: ", acc_pub)
            logging["acc_pub"].append(copy.deepcopy(acc_pub))

        return logging

    def digest_phase(self, i, logging):
        temp_consensus_loss = []
        for j, client in enumerate(self.clients):
            for _ in range(self.consensus_epoch):
                consensus_loss = client.approach_consensus(self.client_optimizers[j])
            #print(f"epoch {i}, client {j}: {consensus_loss}")
            temp_consensus_loss.append(consensus_loss)
        logging["loss_client_consensus"].append(temp_consensus_loss)
        return logging

    def revisit_phase(self, logging):
        for _ in range(self.revisit_epoch):
            loss_local_revisit = self.train_client(public=False)
        logging["loss_client_revisit"].append(loss_local_revisit)
        return logging

    def server_side_training(self, logging):
        # Train a server-side model if it exists
        for _ in range(self.server_training_epoch):
            loss_server_public = self.train_server()
        logging["loss_server_public_dataset"].append(loss_server_public)
        return logging

    def evaluation(self, i, logging):
        acc_on_local_dataset = self.local_score()
        #print(f"epoch={i} acc on local datasets: ", acc_on_local_dataset)
        logging["acc_local"].append(acc_on_local_dataset)
        acc_pub = self.score(self.public_dataloader, self.server_optimizer is None)
        #print(f"epoch={i} acc on public dataset: ", acc_pub)
        logging["acc_pub"].append(copy.deepcopy(acc_pub))
        # evaluation
        if self.test_dataloader is not None:
            acc_test = self.score(
                self.test_dataloader, self.server_optimizer is None
            )
            print(f"epoch={i} acc on test dataset: ", acc_test)
            logging["acc_test"].append(copy.deepcopy(acc_test))

        return logging

    def run(self):
        logging = {
            "loss_client_local_dataset_transfer": [],
            "loss_client_public_dataset_transfer": [],
            "loss_client_consensus": [],
            "loss_client_revisit": [],
            "loss_server_public_dataset": [],
            "acc_local": [],
            "acc_pub": [],
            "acc_test": [],
        }

        logging = self.transfer_phase(logging)

        for i in range(1, self.num_communication + 1):

            self.epoch = i
            self.server.action()
            logging = self.digest_phase(i, logging)
            logging = self.revisit_phase(logging)
            logging = self.server_side_training(logging)
            logging = self.evaluation(i, logging)

            self.custom_action(self)

        return logging

In [23]:
#CNN models
class CNN2L(nn.Module):
    def __init__(self):
        super(CNN2L, self).__init__()
        self.CNN1 = nn.Sequential(nn.Conv2d(in_channels=3,
                                            kernel_size=3, out_channels=128,padding=1),
                                  nn.BatchNorm2d(128),
                                  nn.ReLU(),
                                  nn.Dropout2d(0.2),
                                  nn.AvgPool2d(kernel_size=2, stride=1))
        self.CNN2 = nn.Sequential(nn.Conv2d(in_channels=128,stride=2,
                                            kernel_size=3, out_channels=256),
                                  nn.BatchNorm2d(256),
                                  nn.ReLU(),
                                  nn.Dropout2d(0.2))
        self.FC1 = nn.Linear(57600,10)

    def forward(self, x):
        x = self.CNN1(x)
        x = self.CNN2(x)
        # print(x.shape)
        x = x.view(x.shape[0], -1)
        # print(x.shape)
        x = self.FC1(x)
        return F.softmax(x,dim=1)

class CNN3L(nn.Module):
    def __init__(self, num_classes, n1, n2, n3, dropout_rate):
        super(CNN3L, self).__init__()
        self.CNN1 = nn.Sequential(nn.Conv2d(in_channels=3, #3 channels for CIFAR
                                            kernel_size=3, out_channels=n1,padding=1),
                                  nn.BatchNorm2d(n1),
                                  nn.ReLU(),
                                  nn.Dropout2d(dropout_rate),
                                  nn.AvgPool2d(kernel_size=2, stride=1))
        self.CNN2 = nn.Sequential(nn.Conv2d(in_channels=n1,stride=2,
                                            kernel_size=2, out_channels=n2),
                                  nn.BatchNorm2d(n2),
                                  nn.ReLU(),
                                  nn.Dropout2d(dropout_rate),
                                  nn.AvgPool2d(kernel_size=2))
        self.CNN3 = nn.Sequential(nn.Conv2d(in_channels=n2,stride=2,
                                            kernel_size=3, out_channels=n3),
                                  nn.BatchNorm2d(n3),
                                  nn.ReLU(),
                                  nn.Dropout2d(dropout_rate))
        self.FC1 = nn.Linear(n3 * 9, num_classes)

    def forward(self, x):
        x = self.CNN1(x)
        # print(x.shape)
        x = self.CNN2(x)
        # print(x.shape)
        x = self.CNN3(x)
        # print(x.shape)
        x = x.view(x.shape[0], -1)
        # print(x.shape)
        x = self.FC1(x)
        return F.softmax(x,dim=1)

In [24]:
#ResNet
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.gn1 = nn.GroupNorm(2,out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1)
        self.gn2 = nn.GroupNorm(2,out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out
    
class ResNet20(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet20, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1)
        self.gn1 = nn.GroupNorm(2,16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(16, 16, 3)
        self.layer2 = self.make_layer(16, 32, 3, stride=2)
        self.layer3 = self.make_layer(32, 64, 3, stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(64, num_classes)
        
    def make_layer(self, in_channels, out_channels, block_num, stride=1):
        downsample = None
        if (stride != 1) or (in_channels != out_channels):
            downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1),
                nn.GroupNorm(2,out_channels))
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride, downsample))
        for i in range(1, block_num):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = torch.flatten(out, 1)
        out = self.fc(out)
        return out

In [26]:
models = [CNN2L(), 
          CNN3L(10, 128, 128, 192, 0.2), 
          CNN3L(10,  64,  64,  64, 0.2), 
          CNN3L(10, 128,  64,  64, 0.3), 
          CNN3L(10,  64,  64, 128, 0.4),
          CNN3L(10,  64, 128, 256, 0.2), 
          CNN3L(10,  64, 128, 192, 0.2), 
          CNN3L(10, 128, 192, 256, 0.2), 
          CNN3L(10, 128, 128, 128, 0.3),
          CNN3L(10, 128, 128, 128, 0.3)]

In [ ]:
clients = [
    FedMDClient(models[c].to(device), public_dataloader, output_dim=10, user_id=c, device=device)
    for c in range(num_clients)
]
local_optimizers = [optim.SGD(client.parameters(), lr=lr) for client in clients]

server = FedMDServer(clients, ResNet20().to(device), device=device)

api = FedMD(
    server,
    clients,
    public_dataloader,
    private_dataloaders,
    F.nll_loss,
    local_optimizers,
    test_dataloader,
    num_communication=32,
    device=device
)
log = api.run()

In [ ]:
import matplotlib.pyplot as plt

acc_test = log["acc_test"]
x = list(range(32))
colors = ['red','darkorange','green','blue','purple','brown','magenta','grey','gold','cyan']
legends = []

for i in range(10):
  y = []
  for j in range(32):
    y.append(acc_test[j]["clients_score"][i])
  plt.plot(x, y, color = colors[i], marker='.')
  legends.append("model "+str(i))

plt.xlabel("epochs")
plt.ylabel("test accuracy")
plt.legend(legends, loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()